In [26]:
# Import packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [40]:
# Load datasets
roster = pd.read_csv("D:/Users/John/OneDrive/Documents/SWB/HelpAge International/TZA_2007_SHWALITA_v01_M_CSV/hh_roster.csv")
nonfood = pd.read_csv("D:/Users/John/OneDrive/Documents/SWB/HelpAge International/TZA_2007_SHWALITA_v01_M_CSV/hh_nonfood.csv")
food = pd.read_csv("D:/Users/John/OneDrive/Documents/SWB/HelpAge International/TZA_2007_SHWALITA_v01_M_CSV/hh_frqnonfood.csv")
hh_info = pd.read_csv("D:/Users/John/OneDrive/Documents/SWB/HelpAge International/TZA_2007_SHWALITA_v01_M_CSV/hh_info.csv")

In [3]:
# Create a table with only health expenditures and also scale data to two-week spendings
itemcodes = [2022, 2023, 2024] # Codes of health purchases
nonfood["adjusted"] = 0
for i in range(len(nonfood)):
    if nonfood["recall"][i] == 3: # One-month recall
        nonfood["adjusted"][i] = nonfood["s6value"][i]/2
    elif nonfood["recall"][i] == 4: # One-year recall
        nonfood["adjusted"][i] = nonfood["s6value"][i]/26
nonfood_filtered = nonfood[nonfood["item"].isin(itemcodes)]

D:\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\John\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
# Create a table with every household as a row
unique_hhs = roster.groupby(['id1', 'id2', 'id3', 'id4']).size().reset_index().rename(columns={0:'count'})
n_households = len(unique_hhs)

In [5]:
# Use Simple Household Level to categorize households
unique_hhs["shl"] = 0
for i in range(n_households):
    household = unique_hhs.iloc[i,0:4].ravel()
    roster_household = roster[(roster["id1"] == household[0]) & (roster["id2"] == household[1]) & (roster["id3"] == household[2]) 
           & (roster["id4"] == household[3])]
    n_members = len(roster_household)
    n_60 = len(roster_household[roster_household["s1q5"] >= 60])
    if n_60 == 0:
        unique_hhs["shl"][i] = "NONE"
    elif (n_members == 1) & (n_60 == 1):
        unique_hhs["shl"][i] = "SNGL"
    elif (n_members == 2) & (n_60 > 0):
        unique_hhs["shl"][i] = "DBLE"
    elif (n_members >= 3) & (n_60 > 0):
        unique_hhs["shl"][i] = "OTHR"

D:\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
D:\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
D:\John\Anaconda3\lib\site-packages\i

In [6]:
# Create expenditure totals for the health and nonfood categories
unique_hhs["total_health"] = 0
unique_hhs["total_nonfood"] = 0
for i in range(n_households):
    household = unique_hhs.iloc[i,0:4].ravel()
    health_household = nonfood_filtered[(nonfood_filtered["id1"] == household[0]) & (nonfood_filtered["id2"] == household[1]) 
                                         & (nonfood_filtered["id3"] == household[2]) & (nonfood_filtered["id4"] == household[3])]
    nonfood_household = nonfood[(nonfood["id1"] == household[0]) & (nonfood["id2"] == household[1]) 
                                         & (nonfood["id3"] == household[2]) & (nonfood["id4"] == household[3])]
    total_health = health_household["adjusted"].sum()
    total_nonfood = nonfood_household["adjusted"].sum()
    unique_hhs["total_health"][i] = total_health
    unique_hhs["total_nonfood"][i] = total_nonfood

D:\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [7]:
# Create expenditure totals for the food and frequent category
unique_hhs["total_food"] = 0
for i in range(n_households):
    household = unique_hhs.iloc[i,0:4].ravel()
    food_household = food[(nonfood["id1"] == household[0]) & (food["id2"] == household[1]) 
                                         & (food["id3"] == household[2]) & (food["id4"] == household[3])]
    total_food = food_household["s50value"].sum()
    unique_hhs["total_food"][i] = total_food

D:\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
D:\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [8]:
# Calculate a ratio between health expenditures over total expenditures
unique_hhs["ratio"] = unique_hhs["total_health"] / (unique_hhs["total_nonfood"]+unique_hhs["total_food"])

In [9]:
# Assign whether a household goes beyond the threshold
unique_hhs["10%"] = 0
unique_hhs["25%"] = 0
for i in range(n_households):
    if unique_hhs["ratio"][i] > .1:
        unique_hhs["10%"][i] = 1
    if unique_hhs["ratio"][i] > .25:
        unique_hhs["25%"][i] = 1

D:\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [99]:
# Use bootstrap sampling to compile the final metrics
levels = ["TOTAL", "NONE", "SNGL", "DBLE", "OTHR"]
final_means10 = np.array([])
final_stds10 = np.array([])
final_means25 = np.array([])
final_stds25 = np.array([])
sizes = np.array([])
for i in levels:
    # Select the dataset to sample from
    if i == "TOTAL":
        group = unique_hhs
    else:
        group = unique_hhs[unique_hhs["shl"] == i]
    group_size = len(group)
    means10 = []
    means25 = []
    # Randomly sample the data
    for j in range(1000):
        bootstrap = np.random.randint(0,group_size, size=group_size)
        means10 = np.append(means10,(group["10%"].iloc[bootstrap].mean()))
        means25 = np.append(means25,(group["25%"].iloc[bootstrap].mean()))
    final_means10 = np.append(final_means10,np.mean(means10))
    final_stds10 = np.append(final_stds10,np.std(means10))
    final_means25 = np.append(final_means25,np.mean(means25))
    final_stds25 = np.append(final_stds25,np.std(means25))
    sizes = np.append(sizes,group_size)
# Calculate margins of error
margin10 = 1.96*final_stds10
margin25 = 1.96*final_stds25

In [101]:
# Put the final metrics in a DataFrame
df = pd.DataFrame(list(zip(levels, sizes, final_means10, final_stds10, margin10,
                           final_means25, final_stds25, margin25)),
               columns =["level", "samples", "mean 10%", "SE 10%", "MOE 10%",
                         "mean 25%", "SE 25%", "MOE 25%"])
df.head()

,level,samples,mean 10%,SE 10%,MOE 10%,mean 25%,SE 25%,MOE 25%
0,TOTAL,4032.0,0.347932,0.007410,0.014524,0.101608,0.004750,0.009311
1,NONE,2938.0,0.326551,0.008345,0.016357,0.085036,0.005013,0.009826
2,SNGL,103.0,0.309272,0.044196,0.086623,0.193893,0.039265,0.076959
3,DBLE,148.0,0.501838,0.041504,0.081348,0.265392,0.036438,0.071419
4,OTHR,843.0,0.400730,0.016559,0.032455,0.120346,0.011242,0.022035
